In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue May 11 2021

@author: Zsuzsanna Koczor-Benda, UCL
"""

from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import RectangleSelector
from plotting_functions import get_intensities,broadened_spec_single,get_intens_range,get_target,build_molecules,plot_spectra,write_html

In [2]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import HBox, Layout,VBox

layout = widgets.Layout(width='500px', height='40px',description_width='400px') #set width and height

In [3]:
import pandas as pd
import qgrid

# Load data for all molecules

In [4]:
# Quantities to load:

# wn, R_spec, IR_spec:               wavenumbers, Raman and IR broadened spectra
# freqs:                             normal mode frequencies 
# prod_ints, R_ints, IR_ints:        conversion, Raman, and IR intensties for each normal mode
# smiles, fname:                     smiles codes and database codes of molecules
# P, A, R:                           THz conversion, IR absorption, and Raman target properties for xmin-xmax range


xmin=0     # minimum wavenumber to load (in cm-1)
xmax=4000   # maximum wavenumber to load (in cm-1)

# get intensities and target properties for all molecules
freqs0,prod_ints,R_ints,IR_ints,smiles,fname,P,A,R,R_fre,IR_fre,P_fre=get_intensities(xmin,xmax,sclf=1)

2847 molecules read, maximum number of normal modes: 186
Target properties calculated


# Create histogram and select region of interest

In [5]:
trange=widgets.FloatRangeSlider(
                    min=0,
                    max=3800,
                    value=[30,1000],
                    step=1,
                    continuous_update=False,
                    orientation='horizontal',
                    description='Frequency range (cm-1):',
                    layout=layout,
                    style={'description_width': 'initial'}
                           )
trange

FloatRangeSlider(value=(30.0, 1000.0), continuous_update=False, description='Frequency range (cm-1):', layout=…

In [22]:
%matplotlib widget
tmin=trange.value[0]
tmax=trange.value[1]
print(tmin,tmax)
intens=R_ints
freqs=freqs0*0.98
target_type="R"

intens_range=get_intens_range(freqs,intens,tmin,tmax)      # get intensities in freq range
nb_intens=np.shape(intens_range)[0]*np.shape(intens_range)[1]
intens_max=max(np.reshape(intens_range,nb_intens))
intens_norm=np.reshape(intens_range,nb_intens)/intens_max


def line_select_callback(eclick, erelease):
    'eclick and erelease are the press and release events'
    x1, y1 = eclick.xdata, eclick.ydata
    x2, y2 = erelease.xdata, erelease.ydata

fig, ax = plt.subplots()       # make a new plotting range
ax.set_title("Click and drag to draw a selection area - x only\n")
fig.set_size_inches(8, 6)

# drawtype is 'box' or 'line' or 'none'
rect_select = RectangleSelector(ax, line_select_callback,
                                       drawtype='box', useblit=True,
                                       button=[1, 3],  # don't use middle button
                                       minspanx=5, minspany=5,
                                       spancoords='pixels',
                                       interactive=True)

plt.hist(intens_norm,20,width=0.03) 
plt.yscale('log')
plt.xlabel("{} for range {}-{} cm-1".format(target_type,tmin,tmax))
plt.ylabel("counts")
plt.rcParams.update({'font.size': 16})
plt.tight_layout()
plt.show()

30.0 1000.0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Create table and select mode for calculation

In [24]:
#create default selection (max 50 modes for exemple)
Imin=rect_select.extents[0]*intens_max
Imax=rect_select.extents[1]*intens_max
roi=np.where((intens_range>=Imin) & (intens_range<=Imax))

freq_range=freqs[roi]
R_range=R_ints[roi]
IR_range=IR_ints[roi]
prod_range=prod_ints[roi]

In [25]:
df_types = pd.DataFrame({
    'Molecule' : pd.Series(roi[0],dtype='int'),
    'Mode' : pd.Series(freq_range,dtype='int'),
    #'Frequency' : 
    'Raman activity' : pd.Series(R_range,dtype='f4'),
    'IR intensity' : pd.Series(IR_range,dtype='f2'),
    'Molecular chi(2)' : pd.Series(prod_range,dtype='f4')
})
qgrid_vib = qgrid.show_grid(df_types, show_toolbar=False)

display(qgrid_vib)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [26]:
#create default selection (1st mode for exemple)

mode_button = widgets.Button(description="Mode calculator")
output = widgets.Output()
mode_select=qgrid_vib.get_selected_rows()
if mode_select!=[]:
    f=freq_range[mode_select[0]]
    m=roi[0][mode_select[0]] # molecule number

    display(mode_button, output)

    @output.capture()
    def on_button_clicked(mode_button):
        print(f"Calculations on mode at {f} cm-1 of molecule {m}")
        #print(f'New selected row index: {change.new[0]}')

    mode_button.on_click(on_button_clicked)